In [1]:
#Main model
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from pygame import mixer
from playsound import playsound

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
main_model = load_model('Model\model.h5')

In [5]:
# mixer.init()
# alarm=mixer.Sound('alarm.wav')
capture = cv2.VideoCapture(0)
score = 0 #since counting are fast like in miliseconds.
while True:
    ret,frame = capture.read()
    height,width = frame.shape[:2]
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray,scaleFactor=1.2,minNeighbors=3)
    eyes = eye_cascade.detectMultiScale(gray,scaleFactor=1.2,minNeighbors=3)
    cv2.rectangle(frame,(0,height-50),(200,height),(0,0,0),thickness=cv2.FILLED)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame,pt1=(x,y),pt2=(x+w,y+h),color= (0,0,255),thickness=3)
    
    for (x,y,w,h) in eyes:
        cv2.rectangle(frame,pt1=(x,y),pt2=(x+w,y+h),color= (0,0,255),thickness=3)
        #preprocessing step
        eye = frame[y:y+h,x:x+w]
        eye = cv2.resize(eye,(80,80)) #resize image
        eye = eye/255 #rescaling image by 255
        eye = eye.reshape(80,80,3) #reshape
        eye = np.expand_dims(eye,axis=0)
        #model prediction here
        prediction = main_model.predict(eye)
        if prediction[0][0] > 0.30: #if eyes are close
            cv2.putText(frame,'closed',(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                       thickness=1,lineType=cv2.LINE_AA)
            cv2.putText(frame,'Score'+str(score),(100,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                       thickness=1,lineType=cv2.LINE_AA)
            score = score + 1
            if score>5:
                try:
                    playsound("Car Alert Tone.mp3")
                except:
                    pass
        elif prediction[0][1] > 0.90:
            cv2.putText(frame,'open',(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                       thickness=1,lineType=cv2.LINE_AA)      
            cv2.putText(frame,'Score'+str(score),(100,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                       thickness=1,lineType=cv2.LINE_AA)
            score = score - 1
            if score < 0:
                score = 0
    cv2.imshow('Drowsiness Detection',frame)
    if cv2.waitKey(35) & 0xFF==ord('Q'):
        break
capture.release()
cv2.destroyAllWindows()



    Error 305 for command:
        open "C:\Users\HP\AppData\Local\Temp\PShg7_lg40.mp3"
    Cannot specify extra characters after a string enclosed in quotation marks.

    Error 263 for command:
        close "C:\Users\HP\AppData\Local\Temp\PShg7_lg40.mp3"
    The specified device is not open or is not recognized by MCI.
Failed to close the file: "C:\Users\HP\AppData\Local\Temp\PShg7_lg40.mp3"

    Error 305 for command:
        open "C:\Users\HP\AppData\Local\Temp\PS697rdz4w.mp3"
    Cannot specify extra characters after a string enclosed in quotation marks.

    Error 263 for command:
        close "C:\Users\HP\AppData\Local\Temp\PS697rdz4w.mp3"
    The specified device is not open or is not recognized by MCI.
Failed to close the file: "C:\Users\HP\AppData\Local\Temp\PS697rdz4w.mp3"

    Error 305 for command:
        open "C:\Users\HP\AppData\Local\Temp\PSm90yliuq.mp3"
    Cannot specify extra characters after a string enclosed in quotation marks.

    Error 305 for command:
     